In [1]:
import pandas as pd
import multiprocessing
pd.set_option('display.max_colwidth', None)
import re
import os
os.chdir(r'D:\Хакатон')
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
nltk.download('punkt')
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_score, recall_score, precision_recall_curve
from matplotlib import pyplot as plt
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sunny\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
df = pd.read_excel("ИКРБС_2020-2024_гг..xlsx")

In [3]:
df.fillna('Отсутствует', inplace=True)

In [4]:
df.drop(["№ п/п", "Регистрационный номер"],axis=1,inplace=True)

In [5]:
df['text_data'] = df['Наименование отчёта'] + ' ' + df['Реферативное описание отчета'] + ' ' + df['Ключевые слова']



In [6]:
train_df, test_df = train_test_split(df, test_size=300, train_size=300)

In [7]:
sentence_example = df.iloc[1]["text_data"]
tokens = word_tokenize(sentence_example, language="russian")
tokens_without_punctuation = [i for i in tokens if i not in string.punctuation]
russian_stop_words = stopwords.words("russian")
tokens_without_stop_words_and_punctuation = [i for i in tokens_without_punctuation if i not in russian_stop_words]
snowball = SnowballStemmer(language="russian")
stemmed_tokens = [snowball.stem(i) for i in tokens_without_stop_words_and_punctuation]


In [8]:
print(f"Исходный текст: {sentence_example}")
print("-----------------")
print(f"Токены: {tokens}")
print("-----------------")
print(f"Токены без пунктуации: {tokens_without_punctuation}")
print("-----------------")
print(f"Токены без пунктуации и стоп слов: {tokens_without_stop_words_and_punctuation}")
print("-----------------")
print(f"Токены после стемминга: {stemmed_tokens}")
print("-----------------")

Исходный текст: Исследование технической возможности работы судовых дизелей на сырой нефти Якутского месторождения Определены некоторые свойства сырой нефти Якутского месторождения. Проведено сравнительное испытание показателей работы топливной аппаратуры на нефти и дизельном топливе. Проведены пуски дизеля 2Ч8,5/11 на дизельном топливе и нефти. Проведено сравнение процессов наброса нагрузки на дизеле 2Ч8,5/11, работающем на дизельном топливе и нефти, сняты нагрузочные характеристики дизеля. Сняты винтовые характеристики дизеля 6LI60PNS на дизельном топливе и нефти. ТОПЛИВНАЯ АППАРАТУРА; СУДОВЫЕ ДИЗЕЛИ; СЫРЫЕ НЕФТИ; ДИЗЕЛЬНЫЕ ТОПЛИВА;
-----------------
Токены: ['Исследование', 'технической', 'возможности', 'работы', 'судовых', 'дизелей', 'на', 'сырой', 'нефти', 'Якутского', 'месторождения', 'Определены', 'некоторые', 'свойства', 'сырой', 'нефти', 'Якутского', 'месторождения', '.', 'Проведено', 'сравнительное', 'испытание', 'показателей', 'работы', 'топливной', 'аппаратуры', 'на', 'нефт

In [9]:
snowball = SnowballStemmer(language="russian")
russian_stop_words = stopwords.words("russian")

def tokenize_sentence(sentence: str, remove_stop_words: bool = True):
    tokens = word_tokenize(sentence, language="russian")
    tokens = [i for i in tokens if i not in string.punctuation]
    if remove_stop_words:
        tokens = [i for i in tokens if i not in russian_stop_words]
    tokens = [snowball.stem(i) for i in tokens]
    return tokens

In [10]:
tokenize_sentence(sentence_example)

['исследован',
 'техническ',
 'возможн',
 'работ',
 'судов',
 'дизел',
 'сыр',
 'нефт',
 'якутск',
 'месторожден',
 'определ',
 'некотор',
 'свойств',
 'сыр',
 'нефт',
 'якутск',
 'месторожден',
 'провед',
 'сравнительн',
 'испытан',
 'показател',
 'работ',
 'топливн',
 'аппаратур',
 'нефт',
 'дизельн',
 'топлив',
 'провед',
 'пуск',
 'дизел',
 '2ч8,5/11',
 'дизельн',
 'топлив',
 'нефт',
 'провед',
 'сравнен',
 'процесс',
 'наброс',
 'нагрузк',
 'дизел',
 '2ч8,5/11',
 'работа',
 'дизельн',
 'топлив',
 'нефт',
 'снят',
 'нагрузочн',
 'характеристик',
 'дизел',
 'снят',
 'винтов',
 'характеристик',
 'дизел',
 '6LI60PNS',
 'дизельн',
 'топлив',
 'нефт',
 'топливн',
 'аппаратур',
 'судов',
 'дизел',
 'сыр',
 'нефт',
 'дизельн',
 'топлив']

In [11]:
vectorizer = TfidfVectorizer(tokenizer=lambda x: tokenize_sentence(x, remove_stop_words=True))

In [12]:
features = vectorizer.fit_transform(train_df["text_data"])

D:\Jupiter\Lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [13]:
X = features

In [14]:
y = df['ГРНТИ'][:300]

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [16]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [17]:
y_pred = model.predict(X_test)

In [18]:

accuracy = accuracy_score(y_test, y_pred)
print('Точность:', accuracy)

Точность: 0.0


In [19]:
# Новый текст
new_text = "В данной работе рассматриваются новые методы обработки изображений. Предлагается новый алгоритм, который позволяет улучшить качество изображений и сократить время обработки."

# Преобразование нового текста в вектор признаков
new_vector = vectorizer.transform([new_text])

# Получение предсказания рубрики ГРНТИ
predicted_grnti = model.predict(new_vector)

print("Предсказанная рубрика ГРНТИ:", predicted_grnti)

Предсказанная рубрика ГРНТИ: ['12.09.09: История науки']
